In [1]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3 MB 25 kB/s 
     |████████████████████████████████| 3.8 MB 48.8 MB/s 
     |████████████████████████████████| 503 kB 55.0 MB/s 
     |████████████████████████████████| 50 kB 7.8 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=80207d97d831e469b50e5bf92f96e1b84eb5aef02c1301b2fb50713aff82fe24
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [2]:
!pip install keras==2.0.8

     |████████████████████████████████| 276 kB 8.1 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from google.colab import files
uploaded = files.upload()

Saving dogs_vs_cats_config.py to dogs_vs_cats_config.py


Image To Array Preprocessor

In [5]:
# import packages
from keras.preprocessing.image import img_to_array

class ImageToArrayPreprocessor:
	
	def __init__(self, dataFormat=None):
		# store the image data format
		self.dataFormat = dataFormat

	def preprocess(self, image):
		# apply the Keras utility function that correctly rearranges
		# the dimensions of the image
		return img_to_array(image, data_format=self.dataFormat)

Using TensorFlow backend.


Simple Preprocessor

In [6]:
# import packages
import cv2

class SimplePreprocessor:

    def __init__(self, width, height, inter = cv2.INTER_AREA):
        # store the target image width, height and interpolation
        # method for resizing
        self.width = width
        self.height = height
        self.inter = inter

    def preprocess(self, image):
        # resize the image to a fixed size
        # ignore the aspect ratio
        return cv2.resize(image, (self.width, self.height), interpolation = self.inter)

Patch Preprocessor

In [7]:
# import packages
from sklearn.feature_extraction.image import extract_patches_2d

class PatchPreprocessor:

    def __init__(self, width, height):
        # store the target width and height of the image
        self.width = width
        self.height = height

    def preprocess(self, image):
        # extract a random crop from the image with the target width and height
        return extract_patches_2d(image, (self.height, self.width), max_patches = 1)[0]

 Mean Preprocessor

In [8]:
# import packages
import cv2

class MeanPreprocessor:

    def __init__(self, rMean, gMean, bMean):
        # store the Red, Green, and Blue channel average across a training set
        self.rMean = rMean
        self.gMean = gMean
        self.bMean = bMean

    def preprocess(self, image):
        # split the image into its repsective Red, Green, and Blue
        (B, G, R) = cv2.split(image.astype("float32"))

        # substract the means for each channel
        R -= self.rMean
        G -= self.gMean
        B -= self.bMean

        # merge the channels back together and return the image
        return cv2.merge([B, G, R])

Training Monitor

In [9]:
# import packages
from keras.callbacks import BaseLogger
import matplotlib.pyplot as plt
import numpy as np
import json
import os

class TrainingMonitor(BaseLogger):

    def __init__(self, figPath, jsonPath = None, startAt = 0):
        # store the output path for the figure , the path to the JSON
        # serialized file, and the starting epoch
        super(TrainingMonitor, self).__init__()
        self.figPath = figPath
        self.jsonPath = jsonPath
        self.startAt = startAt

    def on_train_begin(self, logs = {}):
        # initialize the history dictionary
        self.H = {}

        # if the JSON history path exists, load the training history
        if self.jsonPath is not None:
            if os.path.exists(self.jsonPath):
                self.H = json.loads(open(self.jsonPath).read())

                # check to see if a starting epoch was supplied
                if self.startAt > 0:
                    # loop over the entries in the history log and
                    # trim any entries that are past the starting epoch
                    for k in self.H.keys():
                        self.H[k] = self.H[k][:self.startAt]

    def on_epoch_end(self, epoch, logs = {}):
        # loop over the logs and update the loss, accuracy, etc
        # for the entire training process
        for (k, v) in logs.items():
            l = self.H.get(k, [])
            l.append(v)
            self.H[k] = l

        # check to see if the training history should be serialized to file
        if self.jsonPath is not None:
            f = open(self.jsonPath, "w")
            f.write(json.dumps(self.H))
            f.close()

        # ensure at least two epochs have passed before plotting
        # (epoch starts at 0)
        if len(self.H["loss"]) > 1:
            # plot the training loss and accuracy
            N = np.arange(0, len(self.H["loss"]))
            plt.style.use("ggplot")
            plt.figure()
            plt.plot(N, self.H["loss"], label = "train_loss")
            plt.plot(N, self.H["val_loss"], label = "val_loss")
            plt.plot(N, self.H["acc"], label = "train_acc")
            plt.plot(N, self.H["val_acc"], label = "val_acc")
            plt.title("Training Loss and Accuracy [Epoch {}]".format(len(self.H["loss"])))
            plt.xlabel("Epoch #")
            plt.ylabel("Loss/Accuracy")
            plt.legend()

            # save the figure
            plt.savefig(self.figPath)
            plt.close()

HDF5 Dataset Generator

In [10]:
# import packages
from keras.utils import np_utils
# from tensorflow.keras import utils
import numpy as np
import h5py

class HDF5DatasetGenerator:

    def __init__(self, dbPath, batchSize, preprocessors = None, aug = None,
        binarize = True, classes = 2):
        # store the batch size, preprocessors, and data augmentor, whether or not
        # the labels should be binarized, along with the total number of classes
        self.batchSize = batchSize
        self.preprocessors = preprocessors
        self.aug = aug
        self.binarize = binarize
        self.classes = classes

        # open the HDF5 dataset for reading and determine the
        # total number of entries in the database
        self.db = h5py.File(dbPath)
        self.numImages = self.db["labels"].shape[0]

    def generator(self, passes = np.inf):
        # initialize the epoch count
        epochs = 0

        # keep looping infinitely -- the model will stop once we have
        # reach the desired number of epochs
        while epochs < passes:
            # loop over the HDF5 dataset
            for i in np.arange(0, self.numImages, self.batchSize):
                # extract the images and labels from the HDF5 dataset
                images = self.db["images"][i : i + self.batchSize]
                labels = self.db["labels"][i : i + self.batchSize]

                # check to see if the labels should be binarized
                if self.binarize:
                    labels = np_utils.to_categorical(labels, self.classes)

                # check to see if our preprocessors are not None
                if self.preprocessors is not None:
                    # initialize the list of processed images
                    procImages = []

                    # loop over the images
                    for image in images:
                        # loop over the preprocessors and apply each to the image
                        for p in self.preprocessors:
                            image = p.preprocess(image)

                        # update the list of preprocessed images
                        procImages.append(image)

                    # update the images array to be the processed images
                    images = np.array(procImages)

                # if the data augmentator exists, apply it
                if self.aug is not None:
                    (images, labels) = next(self.aug.flow(images, labels,
                        batch_size = self.batchSize))

                # yield a tuple of images and labels
                yield (images, labels)

            # increment the total number of epochs
            epochs += 1

    def close(self):
        # close the database
        self.db.close()

AlexNet

In [13]:
# import packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.regularizers import l2
from keras import backend as K

class AlexNet:
    @staticmethod
    def build(width, height, depth, classes, reg = 0.0002):
        # initialize the model along with the input shape to be
        # "channels last" and channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1

        # if we are using "channel first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

        # Block #1: first CONV => RELU => POOL layer set
        model.add(Conv2D(96, (11, 11), strides = (4, 4), input_shape = inputShape,
            padding = "same", kernel_regularizer = l2(reg)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))
        model.add(MaxPooling2D(pool_size = (3, 3), strides = (2, 2)))
        model.add(Dropout(0.25))

        # Block #2 second CONV => RELU => POOL layer set
        model.add(Conv2D(256, (5, 5), padding = "same", kernel_regularizer = l2(reg)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))
        model.add(MaxPooling2D(pool_size = (3, 3), strides = (2, 2)))
        model.add(Dropout(0.25))

        # Block #3: 3 * (CONV = RELU) => POOL
        model.add(Conv2D(384, (3, 3), padding = "same", kernel_regularizer = l2(reg)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))
        model.add(Conv2D(384, (3, 3), padding = "same", kernel_regularizer = l2(reg)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))
        model.add(Conv2D(256, (3, 3), padding = "same", kernel_regularizer = l2(reg)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))
        model.add(MaxPooling2D(pool_size = (3, 3), strides = (2, 2)))
        model.add(Dropout(0.25))

        # Block #4: first FC => RELU layer set
        model.add(Flatten())
        model.add(Dense(4096, kernel_regularizer = l2(reg)))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        # Block #5: second FC => RELU layer set
        model.add(Dense(4096, kernel_regularizer = l2(reg)))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        # softmax classifiers
        model.add(Dense(classes, kernel_regularizer = l2(reg)))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model

Train Alexnet

In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import packages
import dogs_vs_cats_config as config
# from pipeline.preprocessing import ImageToArrayPreprocessor
# from pipeline.preprocessing import SimplePreprocessor
# from pipeline.preprocessing import PatchPreprocessor
# from pipeline.preprocessing import MeanPreprocessor
# from pipeline.callbacks import TrainingMonitor
# from pipeline.io import HDF5DatasetGenerator
# from pipeline.nn.conv import AlexNet
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import json
import os

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range = 20, zoom_range = 0.15,
    width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.15,
    horizontal_flip = True, fill_mode = "nearest")

# load the RGB means for the training set
means = json.loads(open(config.DATASET_MEAN).read())

# initialize the image preprocessors
sp = SimplePreprocessor(227, 227)
pp = PatchPreprocessor(227, 227)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
iap = ImageToArrayPreprocessor()

# initialize the training and validation dataset generators
trainGen = HDF5DatasetGenerator(config.TRAIN_HDF5, 128, aug = aug,
    preprocessors = [pp, mp, iap], classes = 2)
valGen = HDF5DatasetGenerator(config.VAL_HDF5, 128,
    preprocessors = [sp, mp, iap], classes = 2)

# initialize the optimizer
print("[INFO] compiling model...")
opt = Adam(lr = 1e-3)
model = AlexNet.build(width = 227, height = 227, depth = 3, classes = 2, reg = 0.0002)
model.compile(loss = "binary_crossentropy", optimizer = opt, metrics = ["accuracy"])

# construct the set of callbacks
path = os.path.sep.join([config.OUTPUT_PATH, "{}.png".format(os.getpid())])
callbacks = [TrainingMonitor(path)]

# train the network
model.fit_generator(
    trainGen.generator(),
    steps_per_epoch = trainGen.numImages // 128,
    validation_data = valGen.generator(),
    validation_steps = valGen.numImages // 128,
    epochs = 75,
    max_queue_size = 10,
    callbacks = callbacks, verbose = 1
)

# save the model to file
print("[INFO] serializing model...")
model.save(config.MODEL_PATH, overwrite = True)

# close the HDF5 dataset
trainGen.close()
valGen.close()

In [11]:
from google.colab import files
files.download('example.txt')

1.14.0
